

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICD10_CM.ipynb)




# **ICD10-CM coding**

To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

Import license keys

In [1]:
import os
import json

with open('/content/spark_nlp_for_healthcare.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

SparkNLP Version: 2.6.0
SparkNLP-JSL Version: 2.6.0


Install dependencies

In [2]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
     |████████████████████████████████| 215.7MB 55kB/s 
     |████████████████████████████████| 204kB 46.9MB/s 
     |████████████████████████████████| 133kB 2.9MB/s 
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/2.6.0-8388813d58b67fa25bf9cf603393363af96dba16


Import dependencies into Python

In [3]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl


Start the Spark session

In [4]:
spark = sparknlp_jsl.start(secret)

## 2. Select the Entity Resolver model and construct the pipeline

**NOTE: The mapping below is an example of how ICD10 resolvers work with different NER models. You can choose different combinations  according to your input data and requirements.** 



Select the models:

**ICD10 Entity Resolver models:**

1.   **chunkresolve_icd10cm_clinical**
2.   **chunkresolve_icd10cm_diseases_clinical**
3.   **chunkresolve_icd10cm_injuries_clinical**
4.   **chunkresolve_icd10cm_musculoskeletal_clinical**
5.   **chunkresolve_icd10cm_neoplasms_clinical**
6.   **chunkresolve_icd10cm_puerile_clinical**



For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [6]:
#ner and entity resolver mapping
ner_er_dict = {'chunkresolve_icd10cm_clinical': 'ner_clinical',
              'chunkresolve_icd10cm_diseases_clinical': 'ner_diseases',
              'chunkresolve_icd10cm_injuries_clinical': 'ner_clinical',
              'chunkresolve_icd10cm_musculoskeletal_clinical': 'ner_clinical',
              'chunkresolve_icd10cm_neoplasms_clinical': 'ner_bionlp',
              'chunkresolve_icd10cm_puerile_clinical': 'ner_jsl'}
# ER models are specfic to the codes they are trained on, so we need to filter out entities that will cause noise.
wl_er_dict = {'chunkresolve_icd10cm_clinical': ['PROBLEM'],
              'chunkresolve_icd10cm_diseases_clinical': ['Disease'],
              'chunkresolve_icd10cm_injuries_clinical': ['PROBLEM'],
              'chunkresolve_icd10cm_musculoskeletal_clinical': ['PROBLEM'],
              'chunkresolve_icd10cm_neoplasms_clinical': ['CANCER','PATHOLOGICAL_FORMATION'],
              'chunkresolve_icd10cm_puerile_clinical': ['PROBLEM']}

# Change this to the model you want to use and re-run the cells below.
model = 'chunkresolve_icd10cm_clinical'

Create the pipeline

In [7]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer()\
    .setInputCols(['sentences']) \
    .setOutputCol('tokens')

embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models')\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

ner_model = NerDLModel().pretrained(ner_er_dict[model], 'en', 'clinical/models')\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

#using defined whitelist. You can define your own as well.
ner_chunker = NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunk").setWhiteList(wl_er_dict[model])

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("ner_chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")

entity_resolver = \
    ChunkEntityResolverModel.pretrained(model,"en","clinical/models")\
    .setInputCols("tokens","chunk_embeddings").setOutputCol("resolution")
    
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_chunker,
    chunk_embeddings,
    entity_resolver])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipeline_model = pipeline.fit(empty_df)

light_pipeline = sparknlp.base.LightPipeline(pipeline_model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
chunkresolve_icd10cm_clinical download started this may take some time.
Approximate size to download 166.3 MB
[OK!]


## 3. Create example inputs

In [8]:
# Enter examples as strings in this array
input_list = [
"""The patient is a 5-month-old infant who presented initially on Monday with a cold, cough, and runny nose for 2 days. Mom states she had no fever. Her appetite was good but she was spitting up a lot. She had no difficulty breathing and her cough was described as dry and hacky. At that time, physical exam showed a right TM, which was red. Left TM was okay. She was fairly congested but looked happy and playful. She was started on Amoxil and Aldex and we told to recheck in 2 weeks to recheck her ear. Mom returned to clinic again today because she got much worse overnight. She was having difficulty breathing. She was much more congested and her appetite had decreased significantly today. She also spiked a temperature yesterday of 102.6 and always having trouble sleeping secondary to congestion.""",
             ]

# 4. Run the pipeline

In [9]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)
light_result = light_pipeline.fullAnnotate(input_list[0])

# 5. Visualize

Full Pipeline

In [10]:
result.select(
    F.explode(
        F.arrays_zip('ner_chunk.result', 
                     'ner_chunk.begin',
                     'ner_chunk.end',
                     'ner_chunk.metadata',
                     'resolution.metadata', 'resolution.result')
    ).alias('cols')
).select(
    F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']").alias('begin'),
    F.expr("cols['2']").alias('end'),
    F.expr("cols['3']['entity']").alias('entity'),
    F.expr("cols['4']['resolved_text']").alias('icd10_description'),
    F.expr("cols['5']").alias('icd10_code'),
).toPandas()

,chunk,begin,end,entity,icd10_description,icd10_code
0,"a cold, cough",75,87,PROBLEM,"Chronic obstructive pulmonary disease, unspeci...",J449
1,runny nose,94,103,PROBLEM,Nasal congestion,R0981
2,fever,139,143,PROBLEM,O'nyong-nyong fever,A921
3,difficulty breathing,210,229,PROBLEM,Shortness of breath,R0602
4,her cough,235,243,PROBLEM,Cough,R05
5,fairly congested,365,380,PROBLEM,"Edema, unspecified",R609
6,difficulty breathing,590,609,PROBLEM,Shortness of breath,R0602
7,more congested,625,638,PROBLEM,"Edema, unspecified",R609
8,trouble sleeping,759,774,PROBLEM,"Activity, sleeping",Y9384
9,congestion,789,798,PROBLEM,Nasal congestion,R0981


Light Pipeline

In [11]:
light_result[0]['resolution']

[Annotation(entity, 75, 87, J449, {'chunk': '0', 'all_k_results': 'J449:::R05:::J00:::P800:::L502:::J440:::J45991:::G4483:::T483X5S:::T483X5A:::A3791:::A90:::T483X2A:::T50A16S:::A011:::T483X4A:::T483X3A:::A3701:::T483X4D:::T483X3D:::T483X4S:::A3700:::T50A15D:::T50A13A:::F5221', 'all_k_distances': '1.0191:::1.0633:::1.1106:::1.2013:::1.2018:::1.2401:::1.2861:::1.2883:::1.3233:::1.3330:::1.3475:::1.3516:::1.3541:::1.3666:::1.3689:::1.3723:::1.3785:::1.3813:::1.3822:::1.3826:::1.3846:::1.4072:::1.4106:::1.4257:::1.4269', 'confidence': '0.0533', 'all_k_resolutions': 'Chronic obstructive pulmonary disease, unspecified:::Cough:::Acute nasopharyngitis [common cold]:::Cold injury syndrome:::Urticaria due to cold and heat:::Chronic obstructive pulmonary disease with acute lower respiratory infection:::Cough variant asthma:::Primary cough headache:::Adverse effect of antitussives, sequela:::Adverse effect of antitussives, initial encounter:::Whooping cough, unspecified species with pneumonia:::D